In [2]:
print("ok")

ok


In [3]:
%pwd

'/Users/harshal/Desktop/Euron/ChatBot/MedicalChatbot/research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'/Users/harshal/Desktop/Euron/ChatBot/MedicalChatbot'

In [32]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [23]:
# Extract Data from PDF
def load_pdf_data(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [24]:
extracted_data = load_pdf_data(data='Data/')

In [25]:
len(extracted_data)

4505

In [28]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=80)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [29]:
text_chunks = text_split(extracted_data)
print("Length of text_chunks", len(text_chunks))

Length of text_chunks 23701


In [31]:
# text_chunks

In [35]:
def huggingface_embeddings():
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [36]:
embeddings = huggingface_embeddings()

In [37]:
query_result = embeddings.embed_query("Hello World")
print('length', len(query_result))

length 384


In [39]:
# query_result

In [58]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from pinecone import Pinecone

pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
from pinecone import ServerlessSpec
index_name = "medical-bot"  # change if desired

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [74]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 01 Jul 2025 19:26:28 GMT', 'Content-Type': 'text/plain', 'Content-Length': '12', 'Connection': 'keep-alive', 'x-pinecone-auth-rejected-reason': 'Malformed domain', 'www-authenticate': 'Malformed domain', 'server': 'envoy'})
HTTP response body: Unauthorized


In [48]:
# Load the existing index

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [49]:
docsearch

In [50]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [52]:
retrieved_doc = retriever.invoke("What is Acne ?")

In [70]:
retrieved_doc

[Document(id='897865e0-28bf-42c3-b1d8-2f4e49503f39', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 269.0, 'page_label': '240', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 4505.0}, page_content='Acne is a skin condition that occurs when pores or\nhair follicles become blocked. This blockage allows a\nwaxy material called sebum to collect inside the pores\nor follicles. Normally, sebum flows out onto the skin\n240 GALE ENCYCLOPEDIA OF MEDICINE\nAntiacne drugs'),
 Document(id='cb3e0c54-c720-47f6-b71d-b0cadece0094', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 54.0, 'page_label': '25', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data/Medical_book.pdf', 'total_pages': 4505.0}, page_content='exam before a test can increase the level.\nResources\nPERIODICALS\nMoul,

In [71]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [72]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [69]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)